In [1]:
import tensorflow as tf
from tensorflow import keras

from sklearn.preprocessing import OneHotEncoder

from keras.layers import Input, Dense, Activation
from keras.models import Sequential
from keras.models import Model

import numpy as np
import matplotlib.pyplot as plt
from keras.optimizers import Adam
import pandas as pd

C:\Users\tsmar\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [23]:

#the first thing we need to do is get the data and the labels for the data. 
DataFrameMain = pd.read_csv("C:/Users/tsmar/election_dataframe.csv")
DataFrame = DataFrameMain.select_dtypes(include=["int", "float"])
headers = list(DataFrame.columns.values)

#calculate the split size based on percent of data for train and test
lenTrain = int(0.75*len(DataFrame))
lenTest = int(0.25*len(DataFrame))

#create a mean GOP column, and a mean DEM column (dont worry not inputs into actual training session)
DataFrame["meanGOP"] = DataFrame[["Republicans 2016", "Republicans 2012", "Republicans 2008"]].mean(axis=1) / 100
DataFrame["meanDEM"] = DataFrame[["Democrats 2016", "Democrats 2012", "Democrats 2008"]].mean(axis=1) / 100

#now lets normalize the data so it can actually be used. 
DataFrame["Percent of adults with a bachelor's degree or higher, 2012-2016"] = DataFrame["Percent of adults with a bachelor's degree or higher, 2012-2016"] / 100
DataFrame["Percent of adults completing some college or associate's degree, 2012-2016"] = DataFrame["Percent of adults completing some college or associate's degree, 2012-2016"] / 100
DataFrame["Percent of adults with a high school diploma only, 2012-2016"] = DataFrame["Percent of adults with a high school diploma only, 2012-2016"] / 100
DataFrame["Percent of adults with less than a high school diploma, 2012-2016"] = DataFrame["Percent of adults with less than a high school diploma, 2012-2016"]/100

#more normalization of fields
DataFrame['Median Earnings 2010'] = DataFrame['Median Earnings 2010'] / 100000
DataFrame['White Population'] = DataFrame['White Population'] / 100
DataFrame['African American Population'] = DataFrame['African American Population'] / 100
DataFrame['Native American Population'] = DataFrame['Native American Population'] / 100
DataFrame['Asian American Population'] = DataFrame['Asian American Population'] / 100
DataFrame['Other Race or Races'] = DataFrame['Other Race or Races'] / 100
DataFrame['Latino Population'] = DataFrame['Latino Population'] / 100
DataFrame['Children Under 6 Living in Poverty'] = DataFrame['Children Under 6 Living in Poverty'] /100
DataFrame['Adults 65 and Older Living in Poverty'] = DataFrame['Adults 65 and Older Living in Poverty'] /100
DataFrame['Total Population'] = DataFrame['Total Population'] / 1000000
DataFrame['Preschool Enrollment'] = DataFrame['Preschool Enrollment'] / 100
DataFrame['Poverty Rate'] = DataFrame['Poverty Rate'] / 100
DataFrame['Professional Occupations'] = DataFrame['Professional Occupations'] / 100
DataFrame['Service Occupations'] = DataFrame['Service Occupations'] / 100
DataFrame['Sales and Office Occupations'] = DataFrame['Sales and Office Occupations'] / 100
DataFrame['Sales and Office Occupations'] = DataFrame['Sales and Office Occupations'] / 100
DataFrame['Farming, Fishing, and Forestry Occupations'] = DataFrame['Farming, Fishing, and Forestry Occupations'] /100
DataFrame['Construction Occupations'] = DataFrame['Construction Occupations'] /100
DataFrame['Construction Occupations'] = DataFrame['Construction Occupations'] /100
DataFrame['Manufacturing and Transportation Occupations'] = DataFrame['Manufacturing and Transportation Occupations'] / 100


#pull out relevant fields for training and testing
trainData = DataFrame[["meanGOP", "meanDEM", "Percent of adults with a bachelor's degree or higher, 2012-2016",
                     "Percent of adults completing some college or associate's degree, 2012-2016",
                     "Percent of adults with a high school diploma only, 2012-2016",
                    "Percent of adults with less than a high school diploma, 2012-2016",
                     'Median Earnings 2010','White Population','African American Population',
                     'Native American Population','Asian American Population','Other Race or Races',
                     'Latino Population', 'Children Under 6 Living in Poverty', 'Adults 65 and Older Living in Poverty',
                     'Total Population', 'Preschool Enrollment', 'Poverty Rate', 
                      'Service Occupations', 'Sales and Office Occupations', 'Farming, Fishing, and Forestry Occupations', 
'Construction Occupations', 'Manufacturing and Transportation Occupations']][:lenTrain]

testData = DataFrame[["meanGOP", "meanDEM", "Percent of adults with a bachelor's degree or higher, 2012-2016",
                     "Percent of adults completing some college or associate's degree, 2012-2016",
                     "Percent of adults with a high school diploma only, 2012-2016",
                    "Percent of adults with less than a high school diploma, 2012-2016",
                     'Median Earnings 2010','White Population','African American Population',
                     'Native American Population','Asian American Population','Other Race or Races',
                     'Latino Population', 'Children Under 6 Living in Poverty', 'Adults 65 and Older Living in Poverty',
                     'Total Population', 'Preschool Enrollment', 'Poverty Rate',
                     'Service Occupations', 'Sales and Office Occupations', 'Farming, Fishing, and Forestry Occupations', 
'Construction Occupations', 'Manufacturing and Transportation Occupations']][-1*lenTest:]

trainLabels = []
testLabels = []
#build the train Labels
for index in range(0, len(trainData["meanGOP"])):
    rawData = trainData.values
    j = rawData[index][0]
    k = rawData[index][1]
    if j > k:
        trainLabels.append(np.array([1, 0])) #gop
    else:
        trainLabels.append(np.array([0, 1])) #dem
    
#build the test labels
for index in range(0, len(testData["meanGOP"])):
    rawData = testData.values
    j = rawData[index][0]
    k = rawData[index][1]
    if j > k:
        testLabels.append(np.array([1, 0])) #gop
    else:
        testLabels.append(np.array([0, 1])) #dem


#build the np arrays for passing to keras. 
x_train = np.array(trainData[["Percent of adults with a bachelor's degree or higher, 2012-2016",
                     "Percent of adults completing some college or associate's degree, 2012-2016",
                     "Percent of adults with a high school diploma only, 2012-2016",
                    "Percent of adults with less than a high school diploma, 2012-2016",
                     'Median Earnings 2010','White Population','African American Population',
                     'Native American Population','Asian American Population','Other Race or Races',
                     'Latino Population', 'Children Under 6 Living in Poverty', 'Adults 65 and Older Living in Poverty',
                     'Total Population', 'Preschool Enrollment', 'Poverty Rate','Service Occupations', 'Sales and Office Occupations', 'Farming, Fishing, and Forestry Occupations', 
'Construction Occupations', 'Manufacturing and Transportation Occupations']].values, dtype='float64')
y_train = np.array(trainLabels)

x_test = np.array(testData[["Percent of adults with a bachelor's degree or higher, 2012-2016",
                     "Percent of adults completing some college or associate's degree, 2012-2016",
                     "Percent of adults with a high school diploma only, 2012-2016",
                    "Percent of adults with less than a high school diploma, 2012-2016",
                     'Median Earnings 2010','White Population','African American Population',
                     'Native American Population','Asian American Population','Other Race or Races',
                     'Latino Population', 'Children Under 6 Living in Poverty', 'Adults 65 and Older Living in Poverty',
                     'Total Population', 'Preschool Enrollment', 'Poverty Rate','Service Occupations', 'Sales and Office Occupations', 'Farming, Fishing, and Forestry Occupations', 
'Construction Occupations', 'Manufacturing and Transportation Occupations']].values, dtype='float64')
y_test = np.array(testLabels)

model = Sequential()
model.add(Dense(21, input_dim=21, init='uniform', activation='relu'))
model.add(Dense(2, activation='softmax',init='uniform'))

model.compile(optimizer=Adam(lr=0.02),loss='categorical_crossentropy',metrics=['accuracy'])
scores = model.evaluate(x_test, y_test)
print("--STARTING ACCURACY--\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
model.fit(x_train, y_train, epochs=10, verbose=1)
scores = model.evaluate(x_test, y_test)
print("--ENDING ACCURACY--\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

C:\Users\tsmar\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
C:\Users\tsmar\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\tsmar\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable

C:\Users\tsmar\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\tsmar\Anaconda3\lib\site-packages\ipykernel_launcher.py:115: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(21, input_dim=21, activation="relu", kernel_initializer="uniform")`
C:\Users\tsmar\Anaconda3\lib\site-packages\ipykernel_launcher.py:116: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(2, activation="softmax", kernel_initializer="uniform")`


779/779 [==============================] - 1s 1ms/step
--STARTING ACCURACY--
acc: 42.75%
Epoch 1/10
2337/2337 [==============================] - 1s 524us/step - loss: nan - acc: 0.7848
Epoch 2/10
2337/2337 [==============================] - 0s 58us/step - loss: nan - acc: 0.7920
Epoch 3/10
2337/2337 [==============================] - 0s 55us/step - loss: nan - acc: 0.7920
Epoch 4/10
2337/2337 [==============================] - 0s 51us/step - loss: nan - acc: 0.7920
Epoch 5/10
2337/2337 [==============================] - 0s 49us/step - loss: nan - acc: 0.7920
Epoch 6/10
2337/2337 [==============================] - 0s 47us/step - loss: nan - acc: 0.7920
Epoch 7/10
2337/2337 [==============================] - 0s 51us/step - loss: nan - acc: 0.7920
Epoch 8/10
2337/2337 [==============================] - 0s 51us/step - loss: nan - acc: 0.7920
Epoch 9/10
2337/2337 [==============================] - 0s 49us/step - loss: nan - acc: 0.7920
Epoch 10/10
779/779 [==============================] - 